## Imports

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from IPython import display

import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds

import itertools
import pickle
import math
import re

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec, Word2Vec
import logging

from word2vec_as_MF import Word2vecMF
from functions import *

import time

## Read and preprocess enwik9

In [2]:
# %%time
data = []
with open('data/xaa') as file:
# with open('data/xaa') as file:
    for line in file:
        data+= [line[:-1]]

In [3]:
def wiki_to_wordlist(sentence, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.

    # 3. Convert words to lower case and split them
    words = sentence.split()
    #
    # 4. Optionally remove stop words (false by default)
    
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [4]:
# %%time
# sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")

# sentences += [wiki_to_wordlist(sentence)]
sentences = [sentence.split() for sentence in data]
indices = []

Parsing sentences from training set


In [5]:
# %%time
for i, sentence in enumerate(sentences):
    if not sentence:
        pass
    else:
        indices.append(i)

real_sentences = np.array(sentences)[indices]

In [6]:
'''print('real_sentences', real_sentences[-30:])
print('indices', indices[-30:])
# print('sentences', sentences)'''

"print('real_sentences', real_sentences[-30:])\nprint('indices', indices[-30:])\n# print('sentences', sentences)"

# Gensim

In [7]:
# Create word2vec as matrix factorization model
dimension = 100
model_enwik = Word2vecMF()
model_enwik.data_to_matrices(real_sentences, dimension, 5, 'enwik-200/matrices.npz')

In [ ]:
'''# If the model has been already created, load it from file
model_enwik = Word2vecMF()
model_enwik.load_matrices(from_file='enwik-200/matrices.npz')'''

In [19]:
%%time
skip = Word2Vec(real_sentences, size = dimension, compute_loss=True, min_count= 5, iter=500)

CPU times: user 5min, sys: 1.65 s, total: 5min 1s
Wall time: 1min 57s


In [20]:
skip.get_latest_training_loss()

33622460.0

## Train ro_sgns model starting from SVD of SPPMI

In [10]:
# SVD initialization
SPPMI = np.maximum(np.log(model_enwik.D) - np.log(model_enwik.B), 0)
# print SPPMI
u, s, vt = svds(SPPMI, k=dimension)
C_svd = u.dot(np.sqrt(np.diag(s))).T
W_svd = np.sqrt(np.diag(s)).dot(vt)

/home/yerong/local/Anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


In [11]:
model_enwik.C = C_svd
model_enwik.W = W_svd

model_enwik.save_CW('enwik-200/initializations/SVD_dim200', 0)

In [ ]:
'''# Train the model
start_time = time.time()
opt_experiment(model_enwik,
               mode='PS', 
               d=dimension,
               eta = 5e-5,
               MAX_ITER=10,
               from_iter=0,
               start_from='SVD',
               init=(True, C_svd, W_svd), display=True)
print("--- %s seconds ---" % (time.time() - start_time))'''

In [47]:
# Train the model
start_time = time.time()
opt_experiment(model_enwik,
               mode='AM', 
               d=dimension,
               eta = 1e-5,
               MAX_ITER=50000,
               from_iter=119000,
               start_from='SVD',
               init=(True, C_svd, W_svd), display=True)
print("--- %s seconds ---" % (time.time() - start_time))

enwik-200/AMiter_fromSVD_dim100_step1e-05_factors
Iter #: 119100 loss 953300.610762
Iter #: 119200 loss 953296.7422
Iter #: 119300 loss 953292.88041
Iter #: 119400 loss 953289.025367
Iter #: 119500 loss 953285.177045
Iter #: 119600 loss 953281.335421
Iter #: 119700 loss 953277.500468
Iter #: 119800 loss 953273.672161
Iter #: 119900 loss 953269.850477
Iter #: 120000 loss 953266.035391
Iter #: 120100 loss 953262.226879
Iter #: 120200 loss 953258.424917
Iter #: 120300 loss 953254.629482
Iter #: 120400 loss 953250.84055
Iter #: 120500 loss 953247.058097
Iter #: 120600 loss 953243.282101
Iter #: 120700 loss 953239.512538
Iter #: 120800 loss 953235.749385
Iter #: 120900 loss 953231.992619
Iter #: 121000 loss 953228.242218
Iter #: 121100 loss 953224.49816
Iter #: 121200 loss 953220.760422
Iter #: 121300 loss 953217.028982
Iter #: 121400 loss 953213.30382
Iter #: 121500 loss 953209.584912
Iter #: 121600 loss 953205.872236
Iter #: 121700 loss 953202.165772
Iter #: 121800 loss 953198.465498
Iter

Iter #: 143200 loss 952522.593118
Iter #: 143300 loss 952519.88193
Iter #: 143400 loss 952517.174214
Iter #: 143500 loss 952514.469953
Iter #: 143600 loss 952511.769142
Iter #: 143700 loss 952509.071773
Iter #: 143800 loss 952506.377838
Iter #: 143900 loss 952503.68733
Iter #: 144000 loss 952501.000245
Iter #: 144100 loss 952498.316573
Iter #: 144200 loss 952495.636304
Iter #: 144300 loss 952492.959432
Iter #: 144400 loss 952490.285951
Iter #: 144500 loss 952487.615852
Iter #: 144600 loss 952484.949126
Iter #: 144700 loss 952482.285768
Iter #: 144800 loss 952479.625771
Iter #: 144900 loss 952476.969129
Iter #: 145000 loss 952474.315834
Iter #: 145100 loss 952471.665878
Iter #: 145200 loss 952469.01925
Iter #: 145300 loss 952466.375947
Iter #: 145400 loss 952463.735961
Iter #: 145500 loss 952461.099283
Iter #: 145600 loss 952458.46591
Iter #: 145700 loss 952455.835833
Iter #: 145800 loss 952453.20904
Iter #: 145900 loss 952450.585532
Iter #: 146000 loss 952447.965296
Iter #: 146100 loss

Iter #: 167400 loss 951952.634773
Iter #: 167500 loss 951950.586318
Iter #: 167600 loss 951948.540024
Iter #: 167700 loss 951946.495893
Iter #: 167800 loss 951944.453918
Iter #: 167900 loss 951942.414093
Iter #: 168000 loss 951940.376417
Iter #: 168100 loss 951938.340888
Iter #: 168200 loss 951936.307497
Iter #: 168300 loss 951934.276248
Iter #: 168400 loss 951932.247133
Iter #: 168500 loss 951930.220144
Iter #: 168600 loss 951928.195281
Iter #: 168700 loss 951926.172543
Iter #: 168800 loss 951924.151921
Iter #: 168900 loss 951922.133421
Iter #: 169000 loss 951920.117028
--- 1649.7420620918274 seconds ---


In [ ]:
model_enwik.C = C_svd
model_enwik.W = W_svd
start_time = time.time()
model_enwik.bfgd(d=dimension,from_iter=10000, MAX_ITER=10000, eta=5e-6, display=True,
                 init=(True, C_svd, W_svd), 
                 save=[True, 'dataset'])
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
'''model = Word2Vec(real_sentences, size=200, window=5, min_count=5, workers=4)
fname = 'original'
model.save(fname)
model = Word2Vec.load(fname)  # you can continue training with the loaded model!'''